# GroupfMRI10d - Automated SRActFlow using ANN-inspired representations
#### Control analysis - remove task context and sensory inputs
#### Using ActFlow

#### Takuya Ito
#### 05/15/2019

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools_group_rsa as tools_group
import nibabel as nib
import EmpiricalSRActFlow_ANN_RSA as esr
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"
import time


In [2]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 1.0 Define functions for motor response decodings
## No input from task context

#### Load in FC mappings

In [51]:
for key in fc_input2hidden:
    print(fc_input2hidden[key].shape)


(1660, 2527)
(1660, 2527)
(4833, 2527)
(1499, 2527)


In [50]:
print(fc_logic2hidden.shape)
print(fc_sensory2hidden.shape)
print(fc_motor2hidden.shape)

(39865, 2527)
(16336, 2527)
(15659, 2527)


In [52]:
print(fc_hidden2motorresp.shape)

(2527, 8642)


In [46]:
tools_group = reload(tools_group)
inputtypes = ['color','ori','pitch','constant']
inputkeys = ['RED','VERTICAL','HIGH','CONSTANT']
fc_input2hidden = {}
i = 0
for inputtype in inputtypes:
    fc_input2hidden[inputkeys[i]] = np.real(tools_group.loadGroupActFlowFC(inputtype))
    i += 1
    
# Load rules to hidden FC mappings
fc_logic2hidden = np.real(tools_group.loadGroupActFlowFC('Logic'))
fc_sensory2hidden = np.real(tools_group.loadGroupActFlowFC('Sensory'))
fc_motor2hidden = np.real(tools_group.loadGroupActFlowFC('Motor'))

# Load hidden to motor resp mappings
fc_hidden2motorresp = np.real(tools_group.loadGroupActFlowFC('hidden2out'))

In [4]:
# #threshold FC
# # for key in fc_input2hidden: 
# #     fc_input2hidden[key] = np.multiply(fc_input2hidden[key]>0,fc_input2hidden[key])
# fc_logic2hidden = np.zeros(fc_logic2hidden.shape)
# fc_sensory2hidden = np.zeros(fc_sensory2hidden.shape)
# fc_motor2hidden = np.zeros(fc_motor2hidden.shape)
# fc_hidden2motorresp = np.multiply(fc_hidden2motorresp>0,fc_hidden2motorresp)


In [ ]:
# esr = reload(esr)
# def subjSRActFlowNoTaskContext(subj):
#     print 'Subject', subj
#     obj = esr.EmpiricalActFlow(subj)
#     # Input
#     obj.fc_input2hidden = fc_input2hidden
#     # Rules
#     obj.fc_logic2hidden = fc_logic2hidden
#     obj.fc_sensory2hidden = fc_sensory2hidden
#     obj.fc_motor2hidden = fc_motor2hidden
#     # hidden 2 motor
#     obj.fc_hidden2motorresp = fc_hidden2motorresp
    

#     actflow = obj.generateActFlowPredictionsControl_NoTaskContext(verbose=False, n_inputs=1)
#     del obj
#     return actflow

# global fc_input2hidden
# global fc_logic2hidden
# global fc_sensory2hidden
# global fc_motor2hidden

# timestart = time.time()
# pool = mp.Pool(processes=6)
# results = pool.map_async(subjSRActFlowNoTaskContext,subjNums).get()
# pool.close()
# pool.join()
# timeend = time.time()
# print "time elapsed:", timeend-timestart

# actflow_predictions_notaskcontext = np.asarray(results)

# Load actflow predicted motor responses -- no task context

In [17]:
datafile = '/projects3/SRActFlow/data/results/GroupfMRI/GroupfMRI10d/GroupfMRI10d.h5'
h5f = h5py.File(datafile,'r')
actflow_predictions_notaskcontext = h5f['actflow_predictions_notaskcontext'][:].copy()
h5f.close()

# Load real motor response data

In [18]:
tools_group = reload(tools_group)
nResponses = 2
data_task_rh = np.zeros((len(glasser2),nResponses,len(subjNums)))
data_task_lh = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task_rh[:,:,scount] = np.real(tools_group.loadMotorResponses(subj,hand='Right'))
    data_task_lh[:,:,scount] = np.real(tools_group.loadMotorResponses(subj,hand='Left'))
    scount += 1

tools_group_rsa.py:95: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,0] = data[:,2]
tools_group_rsa.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,1] = data[:,3]
tools_group_rsa.py:98: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,0] = data[:,0] #lmid
tools_group_rsa.py:99: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,1] = data[:,1] #lind


In [19]:
# Set indices for layer-by-layer vertices
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))

target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)

## 1.1 Run across subject decoding on right-hand motor responses

### Compute average activity for each response, for each subject

In [20]:
scount = 0
actflow_rh = np.zeros(data_task_rh.shape)
actflow_lh = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh[target_ind,0,scount] = actflow_predictions_notaskcontext[scount,:,2]
    # RIND
    actflow_rh[target_ind,1,scount] = actflow_predictions_notaskcontext[scount,:,3]
    # LMID
    actflow_lh[target_ind,0,scount] = actflow_predictions_notaskcontext[scount,:,0]
    # LIND
    actflow_lh[target_ind,1,scount] = actflow_predictions_notaskcontext[scount,:,1]


In [34]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

rois = np.asarray([8,9])-1
rois = np.asarray([8,9,52])-1
# rois = np.asarray([9])-1
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# realdata = stats.zscore(data_task_rh[target_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_rh[target_ind,:,:],axis=0).copy()
realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh[roi_ind,:,:].copy()


distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,
                                                          ncvs=ncvs, nproc=nproc)

/usr/local/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice."

## 1.2 Compute statistics

In [22]:
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print 'Activity flow accuracy =', statistics_rh[0,0]
print 'p =', statistics_rh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



Activity flow accuracy = 0.0
p = 1.0
Matched spatial correlation: nan
Mismatched spatial correlation: nan


# 2.0 Run across subject decoding on left-hand motor responses

In [ ]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

rois = np.asarray([188,189]) - 1
# rois = np.asarray([188]) - 1
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# realdata = stats.zscore(data_task_lh[roi_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_lh[roi_ind,:,:],axis=0).copy()
realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh[roi_ind,:,:].copy()


distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,
                                                          ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [ ]:
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print 'Activity flow accuracy =', statistics_lh[0,0]
print 'p =', statistics_lh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



# 2.0
## No input from sensory inputs

#### Load in FC mappings

In [ ]:
# tools_group = reload(tools_group)
# inputtypes = ['color','ori','pitch','constant']
# inputkeys = ['RED','VERTICAL','HIGH','CONSTANT']
# fc_input2hidden = {}
# i = 0
# for inputtype in inputtypes:
#     fc_input2hidden[inputkeys[i]] = np.real(tools_group.loadGroupActFlowFC(inputtype))
#     i += 1
    
# # Load rules to hidden FC mappings
# fc_logic2hidden = np.real(tools_group.loadGroupActFlowFC('Logic'))
# fc_sensory2hidden = np.real(tools_group.loadGroupActFlowFC('Sensory'))
# fc_motor2hidden = np.real(tools_group.loadGroupActFlowFC('Motor'))

# # Load hidden to motor resp mappings
# fc_hidden2motorresp = np.real(tools_group.loadGroupActFlowFC('hidden2out'))

In [ ]:
# #threshold FC
# for key in fc_input2hidden: 
#     fc_input2hidden[key] = np.zeros(fc_input2hidden[key].shape)
# # fc_logic2hidden = np.zeros(fc_logic2hidden.shape)
# # fc_sensory2hidden = np.zeros(fc_sensory2hidden.shape)
# # fc_motor2hidden = np.zeros(fc_motor2hidden.shape)
# fc_hidden2motorresp = np.multiply(fc_hidden2motorresp>0,fc_hidden2motorresp)


In [ ]:
# esr = reload(esr)
# def subjSRActFlowNoSensoryStim(subj):
#     print 'Subject', subj
#     obj = esr.EmpiricalActFlow(subj)
#     # Input
#     obj.fc_input2hidden = fc_input2hidden
#     # Rules
#     obj.fc_logic2hidden = fc_logic2hidden
#     obj.fc_sensory2hidden = fc_sensory2hidden
#     obj.fc_motor2hidden = fc_motor2hidden
#     # hidden 2 motor
#     obj.fc_hidden2motorresp = fc_hidden2motorresp
    

#     actflow = obj.generateActFlowPredictionsControl_NoSensoryStim(verbose=False, n_inputs=1)
#     del obj
#     return actflow

# global fc_input2hidden
# global fc_logic2hidden
# global fc_sensory2hidden
# global fc_motor2hidden

# timestart = time.time()
# pool = mp.Pool(processes=6)
# results = pool.map_async(subjSRActFlowNoSensoryStim,subjNums).get()
# pool.close()
# pool.join()
# timeend = time.time()
# print "time elapsed:", timeend-timestart

# actflow_predictions_nosensory = np.asarray(results)

# Load actflow predicted motor responses -- no sensory info

In [36]:
datafile = '/projects3/SRActFlow/data/results/GroupfMRI/GroupfMRI10d/GroupfMRI10d.h5'
h5f = h5py.File(datafile,'r')
actflow_predictions_nosensory = h5f['actflow_predictions_nosensory'][:].copy()
h5f.close()

In [29]:
# Set indices for layer-by-layer vertices
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))

target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)

## 2.1 Run across subject decoding on right-hand motor responses

### Compute average activity for each response, for each subject

In [37]:
scount = 0
actflow_rh = np.zeros(data_task_rh.shape)
actflow_lh = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh[target_ind,0,scount] = actflow_predictions_nosensory[scount,:,2]
    # RIND
    actflow_rh[target_ind,1,scount] = actflow_predictions_nosensory[scount,:,3]
    # LMID
    actflow_lh[target_ind,0,scount] = actflow_predictions_nosensory[scount,:,0]
    # LIND
    actflow_lh[target_ind,1,scount] = actflow_predictions_nosensory[scount,:,1]


In [38]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

rois = np.asarray([8,9])-1
# rois = np.asarray([9])-1
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# realdata = stats.zscore(data_task_rh[target_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_rh[target_ind,:,:],axis=0).copy()
realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh[roi_ind,:,:].copy()


distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,
                                                          ncvs=ncvs, nproc=nproc)

## 2.2 Compute statistics

In [39]:
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print 'Activity flow accuracy =', statistics_rh[0,0]
print 'p =', statistics_rh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



Activity flow accuracy = 0.729166666667
p = 8.1822771567e-11
Matched spatial correlation: 0.0254517712508
Mismatched spatial correlation: -0.0254517712508


# 2.0 Run across subject decoding on left-hand motor responses

In [44]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

rois = np.asarray([188,189,232]) - 1
rois = np.asarray([188,232]) - 1
# rois = np.asarray([188]) - 1
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# realdata = stats.zscore(data_task_lh[roi_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_lh[roi_ind,:,:],axis=0).copy()
realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh[roi_ind,:,:].copy()


distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,
                                                          ncvs=ncvs, nproc=nproc)
    

## 1.2 Compute statistics

In [45]:
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print 'Activity flow accuracy =', statistics_lh[0,0]
print 'p =', statistics_lh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



Activity flow accuracy = 0.489583333333
p = 0.585668285414
Matched spatial correlation: 0.00728964445583
Mismatched spatial correlation: -0.00728964445583
